# Importing and explaining modules

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
#These are basic library so no need for explaination

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
#This is done to make the model making process smoother nothing else 

In [ ]:
import cv2 
#This library will help in reading image pixels ,manipulating pixels etc. 

In [ ]:
import pathlib
#This library will help in gathering data , will elaborate later. 

In [ ]:
import os 
#This library will help in intraction of notebook with storage , so this will help us to import data. 

In [ ]:
data_dir="/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/train"
data_dir=pathlib.Path(data_dir)
data_dir
#As you can see data_dir has now been converted to a posixpath object which is better for us than simple string str path before 
#This is will help us later for categorization by the use of pathlib library 

In [ ]:
list(data_dir.glob('*/*.jpg'))[:5]
#As you can see glob function has helped us to categorize all files with ".jpg" in train directory
#'*/*' will help us in getting all files with .jpg in it and here */* used when the files are in subdirectory of the directory whoes path is given.

# Preprocessing of Data

In [ ]:
#As the folders are a lot and writting them with brute force will take a lot of time so we will make a for loop containing name of all folders but first we have to make a list with all folder names 
folder_names = [item.name for item in data_dir.iterdir() if item.is_dir()]
#item.name will give the item's name 
#data_dir.iterdir() will iterate over the directory with the path in data_dir which already contains the posixpath to train directory 
#"if item.is_dir()" will check if the item is directory or not 

# Print the folder names
print(folder_names)

In [ ]:
#now we will create the dataset to be used for making a input array for our model
# It will help us in mapping dataset of x and y later 
animals_dataset={}
for i in folder_names:
    if("+" in i):#This is used to remove '+' sign in dataset to make it more readable
        i_new=i.replace('+',' ')
        animals_dataset[f"{i_new}"] = list(data_dir.glob(f'{i}/*'))
    else:
        animals_dataset[f"{i}"] = list(data_dir.glob(f'{i}/*')) 
#This set will help us later in creating x and y for training of model.

I have made sure that we make the dataset before adding the leftout classes in the training set as the training set in pixel play competition does not contain dataset of those classes. 


In [ ]:
#Now we will append remaining classes and also find out the leftout classes. 
remaining_classes=[]
class_path="/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/classes.txt"
#As the text file related to the above path contains all the classes 
with open(class_path, 'r') as file:
    for line in file:
        line=line.split()
        if(line[1] in folder_names):#This make sures that the classes, which are already in folder_names, does not append again. 
            pass
        else:
            print(line[1])
            extra.append(line[1])
            folder_names.append(line[1])

Above shows all the leftout classes. 

In [ ]:
leftout_dir="/kaggle/input/animal-image-dataset-90-different-animals/animals/animals"
#This path leads to dataset related to 7 out of 10 leftout classes. [horse,gorilla,fox,sheep,chimpanzee,squirrel,rhinoceros]
leftout_dir=pathlib.Path(leftout_dir)#Just to convert into posixpath object
for i in extra:
    temp=list(leftout_dir.glob(f'{i}/*'))
    temp.extend(temp)
    temp.extend(temp)#This causes the 7 leftout classes to have 240 images, which helps in data balancing.
    animals_dataset[f'{i}']=temp
rabbit_dir="/kaggle/input/cat-vs-rabbit/train-cat-rabbit/rabbit"
rabbit_dir=pathlib.Path(rabbit_dir)
animals_dataset['rabbit']=list(rabbit_dir.glob('*.jpg'))[:250]
moose_dir="/kaggle/input/moose-and-collie/Moose Data/train"
moose_dir=pathlib.Path(moose_dir)
animals_dataset['moose']=list(moose_dir.glob('*.jpg'))[:250]
collie_dir="/kaggle/input/moose-and-collie/Collie Data/images.cv_xa14kcwlpkyeb4vprlojb/data/train/border_collie"
collie_dir=pathlib.Path(collie_dir)
animals_dataset['collie']=list(collie_dir.glob('*.jpg'))[:250]
#Here i have sliced the dataset to 250 and i think rest is self-explanatory.

In [ ]:
img=cv2.imread('/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/train/antelope/00044.jpg')
img.shape
#This will convert a img from its path to a 3d array that has his value of bgr color.
#Here shape function gives the value of number of pixels in x,y and lastly 3 tells us that it contains different value of rgb combinations 
#This will help us to provide input to the model

In [ ]:
animals_label=[]
for i in folder_names:
    if('+' in i):
        name_new=(i).replace('+',' ')
        animals_label.append(name_new)
        continue 
    else:
        animals_label.append(i)
#It will help in getting the name of animal related to the number and creating a y output array for x input array to train the model.

In [ ]:
x=[]
y=[]
for i in animals_dataset:
    for j in animals_dataset[i]:
        img=cv2.imread(str(j))#Reading its bgr combination and converting it into a 3d array 
        img= cv2.resize(img ,(224,224))#As all the images have different size , we have to resize the image by cv2 library. 
        #Also i have resized to 224 by 224 as densenet base works best for 224 by 224 images.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #This helps in converting bgr to rgb and this is crucial becausing densenet is trained on rgb dataset
        x.append(img)
        y.append(animals_label.index(i))
#Here we are making x and y where x contains the 3d array of its rgb combination and y contains its related number of its animal name .

In [ ]:
x=np.array(x)
y=np.array(y)
#Converting the list into numpy arrays as tensorflow takes arrays
x = preprocess_input(x)

# Model Building

In [ ]:
# Load the DenseNet121 model
densenet_base = DenseNet121(
    include_top=False,         # This is to exclude the dense network as we will be using our own dense network which is better.
    weights='imagenet',        # This tells the pretrained model that we want weights that were pretrained on imagenet dataset.
    input_shape=(224, 224, 3), # Shape of input images.
    pooling='avg',             # Here it helps in reducing the computation and also to flatten the array to input in dense network.
)

In [ ]:
densenet_base.trainable=False
model = Sequential([
  Input(shape=(224,224,3)),#Specifying the input shape
  densenet_base , # Using densenet_base as a convolutional layer.
  layers.Dense(1024, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)), #Here we used L2 regularization to reduce overfitting
  layers.Dropout(0.5), #A dropout layer to reduce overfitting
  layers.Dense(50, activation ='softmax') #A output array with probability of each class.
])
model.compile(optimizer=Adam(learning_rate=1e-4) , #Here we used adam optimizer and a lower learning to get a more accurate model
              loss="sparse_categorical_crossentropy", #self-explanatory
              metrics=['accuracy']) #Metric used is accuracy to tackle underfitting.
model.fit(x, y, epochs=20) 
#As this model is subject to change so i am not putting comments in it 

# Making of csv file 

In [ ]:
test_dir=pathlib.Path("/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/test")
test_images = [item.name for item in test_dir.iterdir()]
test_images.sort()
testing=[]
for i in test_images:
    arr=cv2.imread(f"/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/test/{i}")
    arr = cv2.cvtColor(arr, cv2.COLOR_BGR2RGB)
    arr=cv2.resize(arr,(224,224))
    arr = preprocess_input(arr)
    testing.append(arr)
testing=np.array(testing)
#As explained before in preprocessing of x input , i prefer not to explain twice. 

In [ ]:
predictions=model.predict(testing)
csv_class=[]
for i in range(len(test_images)):
    csv_class.append(animals_label[np.argmax(predictions[i])])#It will help in getting the value with max probability.
df = pd.DataFrame({
    "image_id": test_images ,
    "class" : csv_class }  #Making of csv file
df.to_csv("predictions.csv", index=False) 
